In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import polars as pl

In [4]:
data = pl.read_csv("esewa.csv")
data_pd = pd.read_csv('esewa.csv')

In [6]:
data.head()


user_id,signup_date,user_status,last_active_date,transaction_id,transaction_date,transaction_amount,transaction_type,transaction_category,recurrence_flag
str,str,str,str,str,str,str,str,str,str
"""U00175""","""2024-11-02""","""Active""","""2025-12-21""","""T001036""","""2024-11-05""",null,"""Debit""","""Mobile Recharge""","""No"""
"""U00016""","""2024-11-03""","""Active""","""2025-11-08""","""T000094""","""2024-11-07""","""2419""","""Debit""",null,"""No"""
"""U00004""","""2024-11-04""","""Active""","""2025-07-13""","""T000018""","""2024-11-09""","""1694""","""Debit""","""Phone Bill""","""Yes"""
"""U00125""","""2024-11-03""","""Active""","""2025-10-23""","""T000735""","""2024-11-14""","""NPR 981""","""Debit""","""Internet Bill""","""no"""
"""U00064""","""2024-11-05""","""Active""","""2025-09-08""","""T000353""","""2024-11-15""","""929""","""Debit""","""TV Bill""","""No"""


In [7]:
data_pd.head()

,user_id,signup_date,user_status,last_active_date,transaction_id,transaction_date,transaction_amount,transaction_type,transaction_category,recurrence_flag
0,U00175,2024-11-02,Active,2025-12-21,T001036,2024-11-05,NaN,Debit,Mobile Recharge,No
1,U00016,2024-11-03,Active,2025-11-08,T000094,2024-11-07,2419,Debit,NaN,No
2,U00004,2024-11-04,Active,2025-07-13,T000018,2024-11-09,1694,Debit,Phone Bill,Yes
3,U00125,2024-11-03,Active,2025-10-23,T000735,2024-11-14,NPR 981,Debit,Internet Bill,no
4,U00064,2024-11-05,Active,2025-09-08,T000353,2024-11-15,929,Debit,TV Bill,No


In [16]:
data.select(
    pl.col('user_id') == 'U00175'
)

user_id
bool
true
false
false
false
false
…
false
false
false


In [5]:
def parse_date(date_str):
    
    if pd.isna(date_str) or date_str == '' or str(date_str).strip() == '':
        return pd.NaT
    
    date_str = str(date_str).strip()
    
   
    try:
        return pd.to_datetime(date_str, dayfirst=False)
    except:
        pass
    
    # Try with dayfirst=True for DD/MM/YYYY formats
    try:
        return pd.to_datetime(date_str, dayfirst=True)
    except:
        pass
    
    # Try specific formats manually
    formats = [
        '%Y-%m-%d',      # 2024-01-15
        '%d/%m/%Y',      # 15/01/2024
        '%m/%d/%Y',      # 01/15/2024
        '%Y/%m/%d',      # 2024/01/15
        '%d-%m-%Y',      # 15-01-2024
        '%m-%d-%Y',      # 01-15-2024
        '%Y%m%d',        # 20240115
        '%d/%m/%y',      # 15/01/24
        '%m/%d/%y',      # 01/15/24
    ]
    
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue
    
    
    return pd.NaT

In [6]:
date_columns = ['signup_date', 'last_active_date', 'transaction_date']

print("Parsing dates...")
for col in date_columns:
    if col in data.columns:
       
        data[col] = data[col].apply(parse_date)
        
        
        valid_dates = data[col].notna().sum()
        total_dates = len(data[col])
       

invalid_rows = data[date_columns].isna().any(axis=1).sum()
print(f"\nRows with at least one invalid date: {invalid_rows}")

if invalid_rows > 0:
    print("Sample of rows with invalid dates:")
    print(data[data[date_columns].isna().any(axis=1)][date_columns].head())

# Remove rows with invalid dates
rows_before = len(data)
data = data.dropna(subset=date_columns)
print(f"Removed {rows_before - len(data)} rows with invalid dates")


# Convert to standard format
for col in date_columns:
    data[col] = data[col].dt.strftime('%Y-%m-%d')


Parsing dates...

Rows with at least one invalid date: 0
Removed 0 rows with invalid dates


In [6]:
data['transaction_type'] = data['transaction_type'].fillna('Debit')

In [7]:
data['transaction_type'].value_counts()

transaction_type
Debit    1083
DEBIT      26
debit      23
Dbt        22
DR         16
Name: count, dtype: int64

In [8]:
data['transaction_type'] = data['transaction_type'].str.strip().str.lower()
type_mapping = {
     'debit': 'Debit',
    'DEBIT': 'Debit',
    'debitt': 'Debit',
    'dbt': 'Debit',        
    'dr': 'Debit' 
}
data['transaction_type'] = data['transaction_type'].replace(type_mapping)

In [9]:
data['transaction_category'].unique()

array(['Mobile Recharge', nan, 'Phone Bill', 'Internet Bill', 'TV Bill',
       'Water Bill', 'Electricity Bill', 'Loan Repayment', 'Movie Ticket',
       'School Fee', 'College Fee', 'Air Ticket', 'Government Service',
       'Insurance Premium', 'Bus Ticket', 'CollegeFee', 'ElectricityBill',
       'ELECTRICITY BILL', ' Phone Bill', 'INSURANCE PREMIUM',
       'Air Ticket ', 'Water Bill ', 'MOVIE TICKET', 'PhoneBill',
       'MOBILE RECHARGE', 'tv bill', 'internet bill', 'Movie Ticket ',
       'electricity bill', 'WaterBill', 'Electricity Bill ', 'bus ticket',
       ' Electricity Bill', 'InternetBill', 'Phone_Bill', 'Water_Bill',
       'TV BILL', 'TV_Bill'], dtype=object)

In [10]:
data.drop('recurrence_flag',axis= 1, inplace = True)

In [11]:
data.drop('user_status',axis= 1, inplace = True)
data['transaction_category'].fillna('Other',inplace=True)


/tmp/ipykernel_143/30407009.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['transaction_category'].fillna('Other',inplace=True)


In [12]:
data['transaction_category'].unique()

array(['Mobile Recharge', 'Other', 'Phone Bill', 'Internet Bill',
       'TV Bill', 'Water Bill', 'Electricity Bill', 'Loan Repayment',
       'Movie Ticket', 'School Fee', 'College Fee', 'Air Ticket',
       'Government Service', 'Insurance Premium', 'Bus Ticket',
       'CollegeFee', 'ElectricityBill', 'ELECTRICITY BILL', ' Phone Bill',
       'INSURANCE PREMIUM', 'Air Ticket ', 'Water Bill ', 'MOVIE TICKET',
       'PhoneBill', 'MOBILE RECHARGE', 'tv bill', 'internet bill',
       'Movie Ticket ', 'electricity bill', 'WaterBill',
       'Electricity Bill ', 'bus ticket', ' Electricity Bill',
       'InternetBill', 'Phone_Bill', 'Water_Bill', 'TV BILL', 'TV_Bill'],
      dtype=object)

In [13]:
data.head()

,user_id,signup_date,last_active_date,transaction_id,transaction_date,transaction_amount,transaction_type,transaction_category
0,U00175,2024-11-02,2025-12-21,T001036,2024-11-05,NaN,Debit,Mobile Recharge
1,U00016,2024-11-03,2025-11-08,T000094,2024-11-07,2419,Debit,Other
2,U00004,2024-11-04,2025-07-13,T000018,2024-11-09,1694,Debit,Phone Bill
3,U00125,2024-11-03,2025-10-23,T000735,2024-11-14,NPR 981,Debit,Internet Bill
4,U00064,2024-11-05,2025-09-08,T000353,2024-11-15,929,Debit,TV Bill


In [14]:
string_columns = ['user_id', 'user_status', 'transaction_id', 'transaction_type', 
                  'transaction_category', 'recurrence_flag']

for col in string_columns:
    if col in data.columns:
        data[col] = data[col].astype(str).str.strip()

In [15]:
data['transaction_category'] = data['transaction_category'].str.lower()
category_mapping = {
    # Mobile Recharge variations
    'mobile recharge': 'Mobile Recharge',
    'mobilerecharge': 'Mobile Recharge',
    'mobile_recharge': 'Mobile Recharge',
    
    # Electricity Bill variations
    'electricity bill': 'Electricity Bill',
    'electricitybill': 'Electricity Bill',
    'electricity_bill': 'Electricity Bill',
    
    # Water Bill variations
    'water bill': 'Water Bill',
    'waterbill': 'Water Bill',
    'water_bill': 'Water Bill',
    
    # Phone Bill variations
    'phone bill': 'Phone Bill',
    'phonebill': 'Phone Bill',
    'phone_bill': 'Phone Bill',
    
    # Internet Bill variations
    'internet bill': 'Internet Bill',
    'internetbill': 'Internet Bill',
    'internet_bill': 'Internet Bill',
     # TV Bill variations
    'tv bill': 'TV Bill',
    'tvbill': 'TV Bill',
    'tv_bill': 'TV Bill',
    
    # Movie Ticket variations
    'movie ticket': 'Movie Ticket',
    'movieticket': 'Movie Ticket',
    'movie_ticket': 'Movie Ticket',
    
    # School Fee variations
    'school fee': 'School Fee',
    'schoolfee': 'School Fee',
    'school_fee': 'School Fee',
    
    # College Fee variations
    'college fee': 'College Fee',
    'collegefee': 'College Fee',
    'college_fee': 'College Fee',
    
    # Air Ticket variations
    'air ticket': 'Air Ticket',
    'airticket': 'Air Ticket',
    'air_ticket': 'Air Ticket',
      # Bus Ticket variations
    'bus ticket': 'Bus Ticket',
    'busticket': 'Bus Ticket',
    'bus_ticket': 'Bus Ticket',
    
    # Loan Repayment variations
    'loan repayment': 'Loan Repayment',
    'loanrepayment': 'Loan Repayment',
    'loan_repayment': 'Loan Repayment',
    
    # Insurance Premium variations
    'insurance premium': 'Insurance Premium',
    'insurancepremium': 'Insurance Premium',
    'insurance_premium': 'Insurance Premium',
    
    # Government Service variations
    'government service': 'Government Service',
    'governmentservice': 'Government Service',
    'government_service': 'Government Service',
}
data['transaction_category'] = data['transaction_category'].replace(category_mapping)


In [16]:
data['transaction_category'].unique()

array(['Mobile Recharge', 'other', 'Phone Bill', 'Internet Bill',
       'TV Bill', 'Water Bill', 'Electricity Bill', 'Loan Repayment',
       'Movie Ticket', 'School Fee', 'College Fee', 'Air Ticket',
       'Government Service', 'Insurance Premium', 'Bus Ticket'],
      dtype=object)

In [17]:
data.head()

,user_id,signup_date,last_active_date,transaction_id,transaction_date,transaction_amount,transaction_type,transaction_category
0,U00175,2024-11-02,2025-12-21,T001036,2024-11-05,NaN,Debit,Mobile Recharge
1,U00016,2024-11-03,2025-11-08,T000094,2024-11-07,2419,Debit,other
2,U00004,2024-11-04,2025-07-13,T000018,2024-11-09,1694,Debit,Phone Bill
3,U00125,2024-11-03,2025-10-23,T000735,2024-11-14,NPR 981,Debit,Internet Bill
4,U00064,2024-11-05,2025-09-08,T000353,2024-11-15,929,Debit,TV Bill


In [18]:
data.isna().sum()

user_id                   0
signup_date               0
last_active_date          0
transaction_id            0
transaction_date          0
transaction_amount      112
transaction_type          0
transaction_category      0
dtype: int64

In [19]:
data.shape

(1170, 8)

In [20]:
data.isna().sum()

user_id                   0
signup_date               0
last_active_date          0
transaction_id            0
transaction_date          0
transaction_amount      112
transaction_type          0
transaction_category      0
dtype: int64

In [21]:
data['transaction_amount'] = pd.to_numeric(
    data['transaction_amount'].astype(str).str.replace('NPR', '').str.replace(',', '').str.strip(),
    errors='coerce'
)



In [35]:
user_cat_mean = data.groupby(['user_id', 'transaction_category'])['transaction_amount']
user_cat_mean.describe().head(7)


count    mean         std     min      25%  \
user_id transaction_category                                               
U00001  Electricity Bill        1.0  1979.0         NaN  1979.0  1979.00   
        Internet Bill           0.0     NaN         NaN     NaN      NaN   
        Phone Bill              2.0  2282.5  112.429978  2203.0  2242.75   
        TV Bill                 1.0  1412.0         NaN  1412.0  1412.00   
        Water Bill              1.0  2039.0         NaN  2039.0  2039.00   
        other                   1.0   501.0         NaN   501.0   501.00   
U00002  Bus Ticket              1.0  5061.0         NaN  5061.0  5061.00   

                                 50%      75%     max  
user_id transaction_category                           
U00001  Electricity Bill      1979.0  1979.00  1979.0  
        Internet Bill            NaN      NaN     NaN  
        Phone Bill            2282.5  2322.25  2362.0  
        TV Bill               1412.0  1412.00  1412.0  
        Water Bill            2039.0  2039.00  2039.0  
        other                  501.0   501.00   501.0  
U00002  Bus Ticket            5061.0  5061.00  5061.0

In [140]:
# 3. Fill nulls with user-category mean
user_cat_mean = data.groupby(['user_id', 'transaction_category'])['transaction_amount'].transform('mean')
data['transaction_amount'] = data['transaction_amount'].fillna(user_cat_mean)

# 4. Fallback to category mean (if user never paid that category before)
cat_mean = data.groupby('transaction_category')['transaction_amount'].transform('mean')
data['transaction_amount'] = data['transaction_amount'].fillna(cat_mean)

# 5. Final fallback to overall mean
data['transaction_amount'] = data['transaction_amount'].fillna(data['transaction_amount'].mean())


In [141]:
data.isna().sum()

user_id                 0
signup_date             0
last_active_date        0
transaction_id          0
transaction_date        0
transaction_amount      0
transaction_type        0
transaction_category    0
dtype: int64

In [142]:
data['transaction_amount']

0       1570.521739
1       2419.000000
2       1694.000000
3        981.000000
4        929.000000
           ...     
1165     880.000000
1166    2321.000000
1167    1480.000000
1168    1650.000000
1169    1872.000000
Name: transaction_amount, Length: 1170, dtype: float64

In [161]:
data[data['user_id']=='U00004']

,user_id,signup_date,last_active_date,transaction_id,transaction_date,transaction_amount,transaction_type,transaction_category
2,U00004,2024-11-04,2025-07-13,T000018,2024-11-09,1694.0,Debit,Phone Bill
26,U00004,2024-11-04,2025-07-13,T000015,2024-12-06,807.0,Debit,Electricity Bill
98,U00004,2024-11-04,2025-07-13,T000020,2025-01-06,910.0,Debit,Phone Bill
276,U00004,2024-11-04,2025-07-13,T000019,2025-02-26,1302.0,Debit,Phone Bill
376,U00004,2024-11-04,2025-07-13,T000017,2025-03-23,5903.0,Debit,Loan Repayment
438,U00004,2024-11-04,2025-07-13,T000013,2025-04-09,6960.0,Debit,Government Service
700,U00004,2024-11-04,2025-07-13,T000016,2025-06-20,1959.0,Debit,TV Bill
770,U00004,2024-11-04,2025-07-13,T000014,2025-07-13,1641.0,Debit,other


In [158]:
data.to_csv("Clean_data.csv",index = False)

In [155]:
data['transaction_type'].value_counts()

transaction_type
Debit    1170
Name: count, dtype: int64

In [157]:
data.shape

(1170, 8)

In [7]:
data[data.isna()]

,user_id,signup_date,user_status,last_active_date,transaction_id,transaction_date,transaction_amount,transaction_type,transaction_category,recurrence_flag
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
data[data.isna().any(axis=1)]

,user_id,signup_date,user_status,last_active_date,transaction_id,transaction_date,transaction_amount,transaction_type,transaction_category,recurrence_flag
0,U00175,2024-11-02,Active,2025-12-21,T001036,2024-11-05,NaN,Debit,Mobile Recharge,No
1,U00016,2024-11-03,Active,2025-11-08,T000094,2024-11-07,2419,Debit,NaN,No
6,U00017,2024-11-11,Active,2025-12-14,T000100,2024-11-19,576,NaN,Water Bill,No
7,U00191,2024-11-15,Active,2025-12-16,T001131,2024-11-21,NaN,Debit,Internet Bill,NaN
8,U00124,2024-11-18,Active,2025-12-22,T000725,2024/11/23,NaN,Debit,Electricity Bill,Yes
...,...,...,...,...,...,...,...,...,...,...
1150,U00142,2024-12-28,NaN,2025-12-23,T000832,2025-12-23,1378,Debit,Phone Bill,true
1151,U00183,2025-01-07,Active,2025-12-24,T001090,2025-12-23,1688,Debit,NaN,No
1162,U00147,2025-01-12,Active,2025-12-28,T000855,2025-12-28,NaN,Debit,TV Bill,No
1164,U00039,2025-01-14,Active,2025-12-30,T000219,2025-12-29,1397,NaN,Electricity Bill,No


In [24]:
data["user_id","signup_date"].head()

KeyError: ('user_id', 'signup_date')

In [26]:
import numpy as np
array_3d = np.array([
    [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]],  # Layer 0
    [[13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]] # Layer 1
])

# Accessing a specific element: array[layer][row][column]
# To get the number 15 (Layer 1, Row 0, Column 2)
array_3d[1, 0, 2] 

15